In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd

from pathlib import Path

import pseudopeople as psp

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import alpha

!date
!whoami
!uname -a
!python --version
!pwd

Mon 27 Nov 2023 12:16:21 PM PST
ndbs
Linux int-slurm-sarchive-p0009 5.4.0-156-generic #173-Ubuntu SMP Tue Jul 11 07:25:22 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
Python 3.10.12
/mnt/share/code/ndbs/vivarium_research_prl/model_validation


In [2]:
%load_ext autoreload
%autoreload 2

# Load metadata file

```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow/sample_data/united_states_of_america/2023_08_21_15_58_49/final_results/2023_11_20_14_02_13/pseudopeople_input_data_usa_2.0.0/metadata_proportions.csv
```

# Extra header to create more vertical space so I can see the damn filepath when scrolling -- stupid notebook

In [5]:
yellow_release_path = Path("/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow")
final_results = yellow_release_path / "sample_data/united_states_of_america/2023_08_21_15_58_49/final_results"
metadata_dir = final_results / "2023_11_20_14_02_13/pseudopeople_input_data_usa_2.0.0"
metadata_path = metadata_dir / "metadata_proportions.csv"

metadata_path

PosixPath('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow/sample_data/united_states_of_america/2023_08_21_15_58_49/final_results/2023_11_20_14_02_13/pseudopeople_input_data_usa_2.0.0/metadata_proportions.csv')

In [6]:
df = pd.read_csv(metadata_path)
df

,dataset,state,year,number_of_rows,column,noise_type,proportion
0,decennial_census,WA,2020,10387,date_of_birth,copy_from_household_member,0.793203
1,decennial_census,WA,2030,11217,date_of_birth,copy_from_household_member,0.630917
2,decennial_census,WA,2040,11726,date_of_birth,copy_from_household_member,0.564046
3,decennial_census,WA,3000,33330,date_of_birth,copy_from_household_member,0.657966
4,decennial_census,WA,2020,10387,age,copy_from_household_member,0.793203
...,...,...,...,...,...,...,...
1143,taxes_1040,WA,2037,9457,dependent_4_ssn,copy_from_household_member,0.002221
1144,taxes_1040,WA,2038,9633,dependent_4_ssn,copy_from_household_member,0.002491
1145,taxes_1040,WA,2039,9836,dependent_4_ssn,copy_from_household_member,0.002237
1146,taxes_1040,WA,2040,9897,dependent_4_ssn,copy_from_household_member,0.002526


# Investigate

In [10]:
df[df.year != 3000]

,dataset,state,year,number_of_rows,column,noise_type,proportion
0,decennial_census,WA,2020,10387,date_of_birth,copy_from_household_member,0.793203
1,decennial_census,WA,2030,11217,date_of_birth,copy_from_household_member,0.630917
2,decennial_census,WA,2040,11726,date_of_birth,copy_from_household_member,0.564046
4,decennial_census,WA,2020,10387,age,copy_from_household_member,0.793203
5,decennial_census,WA,2030,11217,age,copy_from_household_member,0.630917
...,...,...,...,...,...,...,...
1142,taxes_1040,WA,2036,9213,dependent_4_ssn,copy_from_household_member,0.002822
1143,taxes_1040,WA,2037,9457,dependent_4_ssn,copy_from_household_member,0.002221
1144,taxes_1040,WA,2038,9633,dependent_4_ssn,copy_from_household_member,0.002491
1145,taxes_1040,WA,2039,9836,dependent_4_ssn,copy_from_household_member,0.002237


In [7]:
df.noise_type.unique()

array(['copy_from_household_member', 'use_nickname'], dtype=object)

In [16]:
proportions = df[df.year != 3000].groupby(['dataset', 'column', 'noise_type'])['proportion']
proportions.describe()

count  \
dataset                    column                 noise_type                          
american_community_survey  age                    copy_from_household_member   23.0   
                           date_of_birth          copy_from_household_member   23.0   
                           first_name             use_nickname                 23.0   
current_population_survey  age                    copy_from_household_member   23.0   
                           date_of_birth          copy_from_household_member   23.0   
                           first_name             use_nickname                 23.0   
decennial_census           age                    copy_from_household_member    3.0   
                           date_of_birth          copy_from_household_member    3.0   
                           first_name             use_nickname                  3.0   
social_security            date_of_birth          copy_from_household_member  122.0   
                           first_name             use_nickname                122.0   
                           middle_name            use_nickname                122.0   
                           ssn                    copy_from_household_member  122.0   
taxes_1040                 age                    copy_from_household_member   23.0   
                           date_of_birth          copy_from_household_member   23.0   
                           dependent_1_first_name use_nickname                 23.0   
                           dependent_1_ssn        copy_from_household_member   23.0   
                           dependent_2_first_name use_nickname                 23.0   
                           dependent_2_ssn        copy_from_household_member   23.0   
                           dependent_3_first_name use_nickname                 23.0   
                           dependent_3_ssn        copy_from_household_member   23.0   
                           dependent_4_first_name use_nickname                 23.0   
                           dependent_4_ssn        copy_from_household_member   23.0   
                           first_name             use_nickname                 23.0   
                           spouse_first_name      use_nickname                 23.0   
                           spouse_last_name       use_nickname                 23.0   
                           spouse_ssn             copy_from_household_member   23.0   
                           ssn                    copy_from_household_member   23.0   
taxes_w2_and_1099          age                    copy_from_household_member   22.0   
                           date_of_birth          copy_from_household_member   22.0   
                           first_name             use_nickname                 22.0   
                           ssn                    copy_from_household_member   22.0   
women_infants_and_children date_of_birth          copy_from_household_member   23.0   
                           first_name             use_nickname                 23.0   

                                                                                  mean  \
dataset                    column                 noise_type                             
american_community_survey  age                    copy_from_household_member  0.645373   
                           date_of_birth          copy_from_household_member  0.645373   
                           first_name             use_nickname                0.605980   
current_population_survey  age                    copy_from_household_member  0.716992   
                           date_of_birth          copy_from_household_member  0.716992   
                           first_name             use_nickname                0.554070   
decennial_census           age                    copy_from_household_member  0.662722   
                           date_of_birth          copy_from_household_member  0.662722   
                           first_name        

In [17]:
_.shape

(34, 8)

In [18]:
proportions.describe()[['mean', 'min', 'max']]

mean  \
dataset                    column                 noise_type                             
american_community_survey  age                    copy_from_household_member  0.645373   
                           date_of_birth          copy_from_household_member  0.645373   
                           first_name             use_nickname                0.605980   
current_population_survey  age                    copy_from_household_member  0.716992   
                           date_of_birth          copy_from_household_member  0.716992   
                           first_name             use_nickname                0.554070   
decennial_census           age                    copy_from_household_member  0.662722   
                           date_of_birth          copy_from_household_member  0.662722   
                           first_name             use_nickname                0.558212   
social_security            date_of_birth          copy_from_household_member  0.526577   
                           first_name             use_nickname                0.603254   
                           middle_name            use_nickname                0.617170   
                           ssn                    copy_from_household_member  0.526577   
taxes_1040                 age                    copy_from_household_member  0.538775   
                           date_of_birth          copy_from_household_member  0.538775   
                           dependent_1_first_name use_nickname                0.054911   
                           dependent_1_ssn        copy_from_household_member  0.112360   
                           dependent_2_first_name use_nickname                0.020137   
                           dependent_2_ssn        copy_from_household_member  0.047432   
                           dependent_3_first_name use_nickname                0.006443   
                           dependent_3_ssn        copy_from_household_member  0.015992   
                           dependent_4_first_name use_nickname                0.001683   
                           dependent_4_ssn        copy_from_household_member  0.004422   
                           first_name             use_nickname                0.568633   
                           spouse_first_name      use_nickname                0.033768   
                           spouse_last_name       use_nickname                0.003293   
                           spouse_ssn             copy_from_household_member  0.048174   
                           ssn                    copy_from_household_member  0.519404   
taxes_w2_and_1099          age                    copy_from_household_member  0.620507   
                           date_of_birth          copy_from_household_member  0.620507   
                           first_name             use_nickname                0.601053   
                           ssn                    copy_from_household_member  0.601658   
women_infants_and_children date_of_birth          copy_from_household_member  0.969974   
                           first_name             use_nickname                0.376379   

                                                                                   min  \
dataset                    column                 noise_type                             
american_community_survey  age                    copy_from_household_member  0.050000   
                           date_of_birth          copy_from_household_member  0.050000   
                           first_name             use_nickname                0.200000   
current_population_survey  age                    copy_from_household_member  0.099034   
                           date_of_birth          copy_from_household_member  0.099034   
                           first_name             use_nickname                0.440000   
decennial_census           age                    copy_from_household_member  0.564046   
                           date

In [20]:
proportions.describe(percentiles=[.025, .975])[['mean', '2.5%', '97.5%']]

mean  \
dataset                    column                 noise_type                             
american_community_survey  age                    copy_from_household_member  0.645373   
                           date_of_birth          copy_from_household_member  0.645373   
                           first_name             use_nickname                0.605980   
current_population_survey  age                    copy_from_household_member  0.716992   
                           date_of_birth          copy_from_household_member  0.716992   
                           first_name             use_nickname                0.554070   
decennial_census           age                    copy_from_household_member  0.662722   
                           date_of_birth          copy_from_household_member  0.662722   
                           first_name             use_nickname                0.558212   
social_security            date_of_birth          copy_from_household_member  0.526577   
                           first_name             use_nickname                0.603254   
                           middle_name            use_nickname                0.617170   
                           ssn                    copy_from_household_member  0.526577   
taxes_1040                 age                    copy_from_household_member  0.538775   
                           date_of_birth          copy_from_household_member  0.538775   
                           dependent_1_first_name use_nickname                0.054911   
                           dependent_1_ssn        copy_from_household_member  0.112360   
                           dependent_2_first_name use_nickname                0.020137   
                           dependent_2_ssn        copy_from_household_member  0.047432   
                           dependent_3_first_name use_nickname                0.006443   
                           dependent_3_ssn        copy_from_household_member  0.015992   
                           dependent_4_first_name use_nickname                0.001683   
                           dependent_4_ssn        copy_from_household_member  0.004422   
                           first_name             use_nickname                0.568633   
                           spouse_first_name      use_nickname                0.033768   
                           spouse_last_name       use_nickname                0.003293   
                           spouse_ssn             copy_from_household_member  0.048174   
                           ssn                    copy_from_household_member  0.519404   
taxes_w2_and_1099          age                    copy_from_household_member  0.620507   
                           date_of_birth          copy_from_household_member  0.620507   
                           first_name             use_nickname                0.601053   
                           ssn                    copy_from_household_member  0.601658   
women_infants_and_children date_of_birth          copy_from_household_member  0.969974   
                           first_name             use_nickname                0.376379   

                                                                                  2.5%  \
dataset                    column                 noise_type                             
american_community_survey  age                    copy_from_household_member  0.205833   
                           date_of_birth          copy_from_household_member  0.205833   
                           first_name             use_nickname                0.273333   
current_population_survey  age                    copy_from_household_member  0.358851   
                           date_of_birth          copy_from_household_member  0.358851   
                           first_name             use_nickname                0.456025   
decennial_census           age                    copy_from_household_member  0.567389   
                           date

In [21]:
proportions.describe()[['50%', '25%', '75%']]

50%  \
dataset                    column                 noise_type                             
american_community_survey  age                    copy_from_household_member  0.625000   
                           date_of_birth          copy_from_household_member  0.625000   
                           first_name             use_nickname                0.600000   
current_population_survey  age                    copy_from_household_member  0.736842   
                           date_of_birth          copy_from_household_member  0.736842   
                           first_name             use_nickname                0.550000   
decennial_census           age                    copy_from_household_member  0.630917   
                           date_of_birth          copy_from_household_member  0.630917   
                           first_name             use_nickname                0.558527   
social_security            date_of_birth          copy_from_household_member  0.568323   
                           first_name             use_nickname                0.600000   
                           middle_name            use_nickname                0.636876   
                           ssn                    copy_from_household_member  0.568323   
taxes_1040                 age                    copy_from_household_member  0.522062   
                           date_of_birth          copy_from_household_member  0.522062   
                           dependent_1_first_name use_nickname                0.058406   
                           dependent_1_ssn        copy_from_household_member  0.113929   
                           dependent_2_first_name use_nickname                0.017546   
                           dependent_2_ssn        copy_from_household_member  0.041461   
                           dependent_3_first_name use_nickname                0.005191   
                           dependent_3_ssn        copy_from_household_member  0.013220   
                           dependent_4_first_name use_nickname                0.001276   
                           dependent_4_ssn        copy_from_household_member  0.003485   
                           first_name             use_nickname                0.566567   
                           spouse_first_name      use_nickname                0.028153   
                           spouse_last_name       use_nickname                0.002843   
                           spouse_ssn             copy_from_household_member  0.039915   
                           ssn                    copy_from_household_member  0.505376   
taxes_w2_and_1099          age                    copy_from_household_member  0.602812   
                           date_of_birth          copy_from_household_member  0.602812   
                           first_name             use_nickname                0.601142   
                           ssn                    copy_from_household_member  0.585362   
women_infants_and_children date_of_birth          copy_from_household_member  0.975410   
                           first_name             use_nickname                0.379032   

                                                                                   25%  \
dataset                    column                 noise_type                             
american_community_survey  age                    copy_from_household_member  0.527778   
                           date_of_birth          copy_from_household_member  0.527778   
                           first_name             use_nickname                0.446023   
current_population_survey  age                    copy_from_household_member  0.692235   
                           date_of_birth          copy_from_household_member  0.692235   
                           first_name             use_nickname                0.520455   
decennial_census           age                    copy_from_household_member  0.597482   
                           date_